In [1]:
import os
import torch as T
import torchvision as TV
import torchaudio as TA
import cv2
import numpy as np
import random
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch import optim
from torch.utils.data import DataLoader, Dataset
import segmentation_models_pytorch as smp
from glob import glob
import albumentations as A
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, confusion_matrix
from pathlib import Path
import segmentation_models_pytorch as smp

In [7]:
T.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [2]:
# ---------------------- DEVICE -----------------------
device = T.device("cuda" if T.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# ---------------------- Paths -----------------------
train_images = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\train_new"
train_masks = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\trainlabel_new"
validation_images = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\validation_new"
validation_masks = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\validationlabel_new"
test_images = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\test_new"
test_masks = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\testlabel_new"

In [4]:
# ---------------------- Simple Transform -----------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ---------------------- Simplified Dataset Class -----------------------
class SimpleSegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, dataset_type="Unknown"):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.dataset_type = dataset_type
        self.image_files = sorted(glob(os.path.join(image_dir, "*.jpg")))
        self.mask_files = sorted(glob(os.path.join(mask_dir, "*.png")))
        self._verify_file_pairs()
        
    def _verify_file_pairs(self):
        if len(self.image_files) != len(self.mask_files):
            raise ValueError(f"Mismatched counts in {self.dataset_type} dataset: {len(self.image_files)} images vs {len(self.mask_files)} masks")
            
        for img_path, mask_path in tqdm(zip(self.image_files, self.mask_files), total=len(self.image_files), desc=f"Verifying {self.dataset_type} File Pairs 🔍"):
            img_name = os.path.splitext(os.path.basename(img_path))[0]
            mask_name = os.path.splitext(os.path.basename(mask_path))[0]
            if img_name != mask_name:
                raise ValueError(f"Filename mismatch in {self.dataset_type} dataset: {img_name} vs {mask_name}")
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # Load image and mask
        img = cv2.cvtColor(cv2.imread(self.image_files[idx]), cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.mask_files[idx], cv2.IMREAD_GRAYSCALE)
        
        # Convert mask to binary (0 or 1)
        mask = (mask > 127).astype(np.uint8)
        
        # Apply transforms
        if self.transform:
            img = self.transform(img)
        else:
            img = transforms.ToTensor()(img)
        
        mask = T.from_numpy(mask).long()
        
        return img, mask

# ---------------------- DataLoaders -----------------------
train_dataset = SimpleSegmentationDataset(train_images, train_masks, transform, "Training")
val_dataset = SimpleSegmentationDataset(validation_images, validation_masks, transform, "Validation")
test_dataset = SimpleSegmentationDataset(test_images, test_masks, transform, "Testing")

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=0, pin_memory=True)  # Changed shuffle to False for validation
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=0, pin_memory=True)   # Changed shuffle to False for testing

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Testing samples: {len(test_dataset)}")

Verifying Testing File Pairs 🔍: 100%|██████████████████████████████████████████| 1200/1200 [00:00<00:00, 85430.96it/s]

Training samples: 1600
Validation samples: 352
Testing samples: 1200


In [5]:
# ---------------------- Model -----------------------
CUSTOM_SAVE_ROOT = Path(r"D:\AAU Internship\Code\UNet-Models")
os.makedirs(CUSTOM_SAVE_ROOT, exist_ok=True)

model = smp.Unet(
    encoder="efficientnet-b5",
    encoder_weights="imagenet",
    encoder_depth=4,
    decoder_use_batchnorm='inplace',
    decoder_attention_type='scse',
    decoder_channels=[256, 128, 64, 32],
    in_channels=3,
    classes=2,
    activation=None,
    center=True,
).to(device)

# ---------------------- Loss Function -----------------------
class FocalTverskyLoss(nn.Module):
    def __init__(self, alpha=0.7, beta=0.3, gamma=0.75, smooth=1e-6):
        super().__init__()
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.smooth = smooth

    def update_hyperparams_by_epoch(self, epoch):
        steps = epoch // 5
        self.alpha = max(0.4, 0.7 - 0.03*steps)
        self.beta = 1 - self.alpha
        self.gamma = min(1.5, 0.5 + 0.1*steps)

    def forward(self, preds, targets):
        targets_one_hot = F.one_hot(targets, num_classes=preds.shape[1]).permute(0, 3, 1, 2).float()
        probs = F.softmax(preds, dim=1)
        dims = (0, 2, 3)
    
        TP = T.sum(probs * targets_one_hot, dims)
        FP = T.sum(probs * (1 - targets_one_hot), dims)
        FN = T.sum((1 - probs) * targets_one_hot, dims)
    
        Tversky = (TP + self.smooth) / (TP + self.alpha * FP + self.beta * FN + self.smooth)
        return T.mean((1 - Tversky) ** self.gamma)

loss_fn = FocalTverskyLoss().to(device)

# ---------------------- Metrics -----------------------
def compute_metrics(preds, targets):
    with T.no_grad():
        pred_labels = T.argmax(preds, dim=1).cpu().numpy().flatten()
        targets = targets.cpu().numpy().flatten()
        ious = []
        for cls in [0, 1]:
            intersection = ((pred_labels == cls) & (targets == cls)).sum()
            union = ((pred_labels == cls) | (targets == cls)).sum()
            ious.append(intersection / (union + 1e-6))
        class_acc = []
        for cls in [0, 1]:
            mask = (targets == cls)
            if mask.sum() > 0:
                class_acc.append((pred_labels[mask] == cls).mean())
        mPA = np.mean(class_acc) * 100
        cm = confusion_matrix(targets, pred_labels)
        TN, FP, FN, TP = cm.ravel()
        return {
            "Accuracy": 100 * accuracy_score(targets, pred_labels),
            "mPA": mPA,
            "Crop IoU": 100 * ious[1],
            "mIoU": 100 * np.mean(ious),
            "Precision": 100 * precision_score(targets, pred_labels, zero_division=0),
            "Recall": 100 * recall_score(targets, pred_labels, zero_division=0),
            "F1-Score": 100 * f1_score(targets, pred_labels, zero_division=0),
            "FNR": 100 * (FN / (FN + TP + 1e-6))
        }

# ---------------------- Training Setup -----------------------
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
MODEL_PATH = CUSTOM_SAVE_ROOT / "best_mPA_model.pth"
best_mPA = -1  # Initialize to negative value for maximization
PRIMARY_METRIC = "mPA"  # Primary metric for model selection

# ---------------------- Training & Validation -----------------------
def TrainUNet(model, dataloader, loss_fn, optimizer, epoch):
    model.train()
    running_loss = 0
    all_preds, all_targets = [], []
    loss_fn.update_hyperparams_by_epoch(epoch)
    loop = tqdm(dataloader, desc=f"Epoch {epoch} [Train]")

    for batch in loop:
        inputs, targets = batch  # Unpack tuple: images, masks
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        all_preds.append(outputs.detach().cpu())
        all_targets.append(targets.detach().cpu())
        loop.set_postfix(loss=loss.item())

    avg_loss = running_loss / len(dataloader)
    metrics = compute_metrics(T.cat(all_preds), T.cat(all_targets))

    T.cuda.empty_cache()
    
    return avg_loss, metrics

def ValidateUNet(model, dataloader, loss_fn):
    model.eval()
    running_loss = 0
    all_preds, all_targets = [], []
    loop = tqdm(dataloader, desc="Validating")

    with T.no_grad():
        for batch in loop:
            inputs, targets = batch  # Unpack tuple: images, masks
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            running_loss += loss.item()
            all_preds.append(outputs.detach().cpu())
            all_targets.append(targets.detach().cpu())
            loop.set_postfix(loss=loss.item())

    avg_loss = running_loss / len(dataloader)
    metrics = compute_metrics(T.cat(all_preds), T.cat(all_targets))
    
    T.cuda.empty_cache()

    return avg_loss, metrics

# ---------------------- Main Training -----------------------
num_epochs = 50
for epoch in range(1, num_epochs + 1):
    train_loss, train_metrics = TrainUNet(model, train_dataloader, loss_fn, optimizer, epoch)
    val_loss, val_metrics = ValidateUNet(model, val_dataloader, loss_fn)

    # Save model if mPA improves
    if val_metrics[PRIMARY_METRIC] > best_mPA:
        best_mPA = val_metrics[PRIMARY_METRIC]
        T.save(model.state_dict(), str(MODEL_PATH))
        print(f"✅ New best {PRIMARY_METRIC}: {best_mPA:.2f}% | Saved to: {MODEL_PATH}")

    # Print epoch summary
    print(f"\n📊 Epoch {epoch} Summary:")
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    for k, v in val_metrics.items():
        print(f"{k}: {v:.2f}%")

    T.cuda.empty_cache()

# ---------------------- Final Report -----------------------
print(f"\n🎯 === Best Model Summary ===")
print(f"Best {PRIMARY_METRIC}: {best_mPA:.2f}% → {MODEL_PATH}")

# ---------------------- Testing -----------------------
print("\n🧪 === Testing Best Model ===")
model.load_state_dict(T.load(str(MODEL_PATH)))
test_loss, test_metrics = ValidateUNet(model, test_dataloader, loss_fn)
print(f"\n📌 Best {PRIMARY_METRIC} Model Test Results:")
for k, v in test_metrics.items():
    print(f"{k}: {v:.2f}%")

Validating: 100%|██████████████████████████████████████████████████████████| 88/88 [00:21<00:00,  4.12it/s, loss=0.155]


✅ New best mPA: 98.27% | Saved to: D:\AAU Internship\Code\UNet-Models\best_mPA_model.pth

📊 Epoch 1 Summary:
Train Loss: 0.3262 | Val Loss: 0.1638
Accuracy: 99.21%
mPA: 98.27%
Crop IoU: 92.77%
mIoU: 95.95%
Precision: 95.43%
Recall: 97.09%
F1-Score: 96.25%
FNR: 2.91%


Validating: 100%|██████████████████████████████████████████████████████████| 88/88 [00:19<00:00,  4.45it/s, loss=0.123]


✅ New best mPA: 98.48% | Saved to: D:\AAU Internship\Code\UNet-Models\best_mPA_model.pth

📊 Epoch 2 Summary:
Train Loss: 0.1515 | Val Loss: 0.1247
Accuracy: 99.36%
mPA: 98.48%
Crop IoU: 94.04%
mIoU: 96.66%
Precision: 96.49%
Recall: 97.37%
F1-Score: 96.93%
FNR: 2.63%


Validating: 100%|██████████████████████████████████████████████████████████| 88/88 [00:19<00:00,  4.43it/s, loss=0.121]


...


📊 Epoch 49 Summary:
Train Loss: 0.0012 | Val Loss: 0.0038
Accuracy: 99.38%
mPA: 98.63%
Crop IoU: 94.25%
mIoU: 96.78%
Precision: 96.40%
Recall: 97.68%
F1-Score: 97.04%
FNR: 2.32%


Validating: 100%|████████████████████████████████████████████████████████| 88/88 [00:19<00:00,  4.51it/s, loss=0.00202]



📊 Epoch 50 Summary:
Train Loss: 0.0008 | Val Loss: 0.0021
Accuracy: 99.46%
mPA: 98.83%
Crop IoU: 94.99%
mIoU: 97.20%
Precision: 96.83%
Recall: 98.04%
F1-Score: 97.43%
FNR: 1.96%

🎯 === Best Model Summary ===
Best mPA: 98.88% → D:\AAU Internship\Code\UNet-Models\best_mPA_model.pth

🧪 === Testing Best Model ===


Validating: 100%|██████████████████████████████████████████████████████| 300/300 [01:12<00:00,  4.14it/s, loss=0.00361]



📌 Best mPA Model Test Results:
Accuracy: 99.47%
mPA: 98.85%
Crop IoU: 94.49%
mIoU: 96.95%
Precision: 96.27%
Recall: 98.08%
F1-Score: 97.16%
FNR: 1.92%


In [12]:
# ---------------------- Mask Generation -----------------------
from pathlib import Path
from tqdm import tqdm

MASK_FOLDER_NAME = "Crop_Masks"  # User-specified folder name for saving masks

# Create output folder in the notebook's directory
MASK_OUTPUT_DIR = Path.cwd() / MASK_FOLDER_NAME
MASK_OUTPUT_DIR.mkdir(exist_ok=True)  # Create folder if it doesn't exist

def save_segmentation_masks(model, train_dataloader, val_dataloader, test_dataloader, model_path, output_dir, device):
    model.load_state_dict(T.load(str(model_path)))
    model.eval()
    dataloaders = [train_dataloader, val_dataloader, test_dataloader]
    mask_counter = 1
    
    with T.no_grad():
        for dataloader in tqdm(dataloaders, desc="Processing datasets"):
            for batch in tqdm(dataloader, desc="Processing batch", leave=False):
                inputs, _ = batch  # Unpack tuple: images, masks (ignore masks)
                inputs = inputs.to(device)
                batch_size = inputs.size(0)
                
                # Generate predictions
                outputs = model(inputs)
                pred_labels = T.argmax(outputs, dim=1).cpu().numpy()  # Shape: (batch_size, H, W)
                
                # Save each mask in the batch
                for i in range(batch_size):
                    mask = pred_labels[i]  # Shape: (H, W), values 0 or 1
                    mask = (mask * 255).astype(np.uint8)  # Convert to 0, 255 for PNG
                    mask_path = str(output_dir / f"{mask_counter}c_mask.png")
                    cv2.imwrite(mask_path, mask)
                    mask_counter += 1
    
    print(f"\n🎉 Saved {mask_counter - 1} segmentation masks to {output_dir}")

print("\n🖼️ Generating and saving segmentation masks...")
save_segmentation_masks(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    test_dataloader=test_dataloader,
    model_path=CUSTOM_SAVE_ROOT / "best_mPA_model.pth",
    output_dir=MASK_OUTPUT_DIR,
    device=device
)
T.cuda.empty_cache()


🖼️ Generating and saving segmentation masks...


Processing datasets: 100%|███████████████████████████████████████████████████████████████| 3/3 [03:06<00:00, 62.05s/it]


🎉 Saved 3152 segmentation masks to D:\AAU Internship\Code\Crop_Masks
